# **Preprocessing File** 
**For Specific Storms with Smaller Datasets**

Installtion of required packages

In [ ]:
! pip install radiant_mlhub
! pip install -q livelossplot
! pip install wandb
! pip install pycm
! pip install nb-black
! echo $?

%load_ext nb_black

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 321 kB/s 
     |████████████████████████████████| 137 kB 10.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 793 kB 5.4 MB/s 
     |████████████████████████████████| 381 kB 9.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fo

<IPython.core.display.Javascript object>

Import Packages

In [ ]:
import os

# import the api key for the ML Radient Package
os.environ['MLHUB_API_KEY'] = 'c0557d3a6fcffba37ad3002563d2de88e7a32f8e77dc62b4d11c7c12b437152f'

# Import Packages
import json
import re
from glob import glob
import tarfile

import numpy as np
import pandas as pd
from pathlib import Path
from radiant_mlhub import Dataset, Collection, client

from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.image as mpimage
import numpy as np
import pandas as pd
import seaborn as sns
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms


import tkinter as tk
from tkinter import *
from tkinter import filedialog

try:
    from PIL import Image
except ImportError:
    import Image
from PIL import Image as imim

import shutil

pd.set_option("max_colwidth", 80)

<IPython.core.display.Javascript object>

Extracting Files

In [ ]:
sup_paths = ["/root/Downloads/nasa_tropical_storm_competition_surprise_storm_source.tar.gz", "/root/Downloads/nasa_tropical_storm_competition_surprise_storm_labels.tar.gz", "/root/Downloads/nasa_tropical_storm_competition_surprise_storm_metrics.tar.gz"]
for archive_path in sup_paths:
    print(f'Extracting {archive_path}...')
    with tarfile.open(archive_path) as tfile:
        tfile.extractall(path=download_dir)
print('Done')

Extracting /root/Downloads/nasa_tropical_storm_competition_surprise_storm_source.tar.gz...
Extracting /root/Downloads/nasa_tropical_storm_competition_surprise_storm_labels.tar.gz...
Extracting /root/Downloads/nasa_tropical_storm_competition_surprise_storm_metrics.tar.gz...
Done


<IPython.core.display.Javascript object>

Creating Correct Directories

In [ ]:
def refresh_folder(path):
  if os.path.exists(path):
    shutil.rmtree(path)
    os.mkdir(path)
  else:
    os.mkdir(path)

<IPython.core.display.Javascript object>

Extract Images to the Directory

In [ ]:
from operator import length_hint
def copy_jpg(leng, name, bias, old_path):
  old_path = old_path+str(name)
  new_path = "/root/Downloads/train_source/"
  refresh_folder(new_path)
  for i in range(bias):
    new_path = "/root/Downloads/train_source/"+str(name)+str(i)
    if not os.path.exists(new_path):
      os.mkdir(new_path)
    for j in range(leng-i):
      if ((j+i+1) < 10):
         old_p = old_path+"_00"+str(j+i+1)+"/image.jpg"
      elif ((j+i+1) < 100):
        old_p = old_path+"_0"+str(i+j+1)+"/image.jpg"
      elif ((j+i+1) < 1000):
        old_p = old_path+"_"+str(i+j+1)+"/image.jpg"
      new_p = new_path+"/"+str(j)+".jpg"
      shutil.copyfile(str(old_p), str(new_p))

<IPython.core.display.Javascript object>

In [ ]:
old_sup="/root/Downloads/nasa_tropical_storm_competition_surprise_storm_source/nasa_tropical_storm_competition_surprise_storm_source_"
copy_jpg(95, "sss", 5,old_sup)

<IPython.core.display.Javascript object>

Function to Convert Images to JPG

In [ ]:
def convertjpg(jpgfile,outdir,width=64,height=64):
    img=Image.open(jpgfile)
    try:
        new_img = img.resize((width, height), Image.BILINEAR)
        if new_img.mode == 'P':
            new_img = new_img.convert("RGB")
        if new_img.mode == 'RGBA':
            new_img = new_img.convert("RGB")
        new_img.save(outdir)
    except Exception as e:
        print(e)

def read_images(path):
    data = []
    for img in os.listdir(path):
      try:
          img_arr = mpimage.imread(os.path.join(path, img))
          # resized_arr = np.resize(img_arr, image_size) # Reshaping images to preferred size
          data.append(img_arr.tolist())
      except Exception as e:
        print(e)
    return np.array(data, dtype=np.float32)

# standardise the data
def apply_standardization(X): # define an standardisation function
    X /= 255.
    return X


<IPython.core.display.Javascript object>

Function to Split the Data

In [ ]:
import glob

def data_split(storm_name, total_size = 600, input_size = 5, output_size =5, bias = 5, ratio = 0.8):
  total_data = []
  train_set_path = '/root/Downloads/train_set'
  train_label_path = '/root/Downloads/train_label'
  test_set_path = '/root/Downloads/test_set'
  test_label_path = '/root/Downloads/test_label'
  original_path = "/root/Downloads/train_source/"+str(storm_name)
  for b in range(bias):
    refresh_folder(train_set_path)
    refresh_folder(train_label_path)
    refresh_folder(test_set_path)
    refresh_folder(test_label_path)
    original_path = "/root/Downloads/train_source/"+str(storm_name)+str(b)

    files = glob.glob(os.path.join(original_path, '*.jpg'))
    for file in files:
      convertjpg(file,file,width=64,height=64)

    train_size = int((total_size * ratio)/(input_size+output_size))

    for i in range(train_size):
      new_path = train_set_path+"/seq%s"%(i+1)
      new_label_path = train_label_path+"/seq%s"%(i+1)
      refresh_folder(new_path)
      refresh_folder(new_label_path)

      for j in range(input_size):
        numb = int(i*(input_size+output_size)+j)
        old_p = original_path+"/%s.jpg"%(str(numb))
        new_p = new_path+"/%s.jpg"%(str(j))
        shutil.copyfile(str(old_p), str(new_p))
      for k in range(output_size):
        numb = int(i*(input_size+output_size)+input_size+k)
        old_p = original_path+"/%s.jpg"%(str(numb))
        new_p = new_label_path+"/%s.jpg"%(str(input_size+k))
        shutil.copyfile(str(old_p), str(new_p))

    before = int(total_size*ratio)
    test_size = math.ceil((total_size * (1-ratio))/(input_size+output_size))

    for i in range(test_size):
      new_path = test_set_path+"/ts_seq%s"%(i+1)
      new_label_path = test_label_path+"/ts_seq%s"%(i+1)
      refresh_folder(new_path)
      refresh_folder(new_label_path)
      for j in range(input_size):
        numb = int(i*(input_size+output_size)+j+before)
        old_p = original_path+"/%s.jpg"%(str(numb))
        new_p = new_path+"/%s.jpg"%(str(j))
        shutil.copyfile(str(old_p), str(new_p))
      for k in range(output_size):
        numb = int(i*(input_size+output_size)+input_size+k+before)
        old_p = original_path+"/%s.jpg"%(str(numb))
        new_p = new_label_path+"/%s.jpg"%(str(input_size+k))
        shutil.copyfile(str(old_p), str(new_p))
    
    this_data = []
    for i in range(train_size):
      dd = read_images('/root/Downloads/train_set/seq%s'%(str(i+1)))
      ll = read_images('/root/Downloads/train_label/seq%s'%(str(i+1)))
      toget = np.vstack((dd, ll))
      this_data.append(toget)
    this_data = np.array(this_data, dtype=float)
    print(this_data.shape)
    if (b==0):
      total_data = this_data
    else:
      total_data = np.vstack((total_data, this_data))
  total_data = np.array(total_data, dtype=float)
  print(total_data.shape)
  return total_data

<IPython.core.display.Javascript object>

Apply the Data_split Function 

In [ ]:
sup_data = data_split("sss", total_size = 90, input_size = 5, output_size =5, bias = 5, ratio = 1)

(9, 10, 64, 64)
(9, 10, 64, 64)
(9, 10, 64, 64)
(9, 10, 64, 64)
(9, 10, 64, 64)
(45, 10, 64, 64)


<IPython.core.display.Javascript object>

In [ ]:
print(sup_data[0][0])
model_data = torch.tensor(apply_standardization(sup_data)).float()
print(model_data.shape)
print(model_data[0][0])

[[ 10.   9.   8. ... 108. 103. 102.]
 [ 10.   9.   8. ... 112. 114. 124.]
 [ 10.  10.   9. ... 114. 116. 126.]
 ...
 [103.  96.  85. ... 139. 128. 121.]
 [ 85.  86.  86. ... 142. 128. 121.]
 [ 94.  88.  84. ... 147. 131. 122.]]
torch.Size([45, 10, 64, 64])
tensor([[0.0392, 0.0353, 0.0314,  ..., 0.4235, 0.4039, 0.4000],
        [0.0392, 0.0353, 0.0314,  ..., 0.4392, 0.4471, 0.4863],
        [0.0392, 0.0392, 0.0353,  ..., 0.4471, 0.4549, 0.4941],
        ...,
        [0.4039, 0.3765, 0.3333,  ..., 0.5451, 0.5020, 0.4745],
        [0.3333, 0.3373, 0.3373,  ..., 0.5569, 0.5020, 0.4745],
        [0.3686, 0.3451, 0.3294,  ..., 0.5765, 0.5137, 0.4784]])
torch.Size([45, 1, 10, 64, 64])


<IPython.core.display.Javascript object>

In [ ]:
from matplotlib import test
var_path = os.path.join("/root/Downloads/TEST.npy")
np_var = model_data.data.cpu().numpy()
np.save(var_path, np_var)
print(np_var.shape)

(45, 10, 64, 64)


<IPython.core.display.Javascript object>